In [9]:
import csv

# Initialize total P&L
total_pl = 0.0

# Read the CSV file
with open('../output/rh_statement.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        # Check if the row has enough columns
        if len(row) < 9:
            print(f"Skipping row with insufficient columns: {row}")
            continue
                
        # Check if the description contains "ACH"
        if "ACH" in row[4]:
            print(f"Skipping ACH transaction: {row[4]}")
            continue

        # # Check if the description contains "CUSIP" which is a stock purchase
        # if "CUSIP:" in row[4]:
        #     print(f"Skipping CUSIP transaction: {row[4]}")
        #     continue

          # Check if the description is empty ""
        if "" == row[3]:
            print(f"Skipping RH Gold transaction: {row[4]}")
            continue
        
        # Replace empty Price and Amount cells with '0'
        row[7] = row[7] if row[7] else '0'  # Price column
        row[8] = row[8] if row[8] else '0'  # Amount column
        
        amount = row[8].replace('$', '').replace(',', '')  # Clean the amount
        print(f"Processing amount: {amount}")  # Debug print
        if '(' in amount:  # Check for loss
            amount = amount.replace('(', '').replace(')', '')  # Remove parentheses
            total_pl -= float(amount)  # Subtract from total
            print(f"Subtracted {amount}, new total: {total_pl}")  # Debug print
        else:
            total_pl += float(amount)  # Add to total
            print(f"Added {amount}, new total: {total_pl}")  # Debug print

print(f'Total Profit and Loss: ${total_pl:.2f}')

Processing amount: 0
Added 0, new total: 0.0
Processing amount: (75.03)
Subtracted 75.03, new total: -75.03
Skipping RH Gold transaction: Gold Subscription Fee
Skipping RH Gold transaction: Interest Payment
Skipping RH Gold transaction: Gold Deposit Boost Payment
Processing amount: 0
Added 0, new total: -75.03
Processing amount: (265.03)
Subtracted 265.03, new total: -340.05999999999995
Processing amount: 0.95
Added 0.95, new total: -339.10999999999996
Processing amount: (217.03)
Subtracted 217.03, new total: -556.14
Processing amount: 63.95
Added 63.95, new total: -492.19
Processing amount: 178.95
Added 178.95, new total: -313.24
Processing amount: (81.03)
Subtracted 81.03, new total: -394.27
Processing amount: (251.03)
Subtracted 251.03, new total: -645.3
Processing amount: 539.94
Added 539.94, new total: -105.3599999999999
Processing amount: (417.03)
Subtracted 417.03, new total: -522.3899999999999
Processing amount: (9067.03)
Subtracted 9067.03, new total: -9589.42
Processing amoun

In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('../output/rh_statement.csv', on_bad_lines='skip')

# Remove rows with an empty 'Instrument' column
df = df[df['Instrument'].notna() & (df['Instrument'] != '')]

# Remove rows with "CUSIP" in the 'Description' column
df = df[~df['Description'].str.contains('CUSIP', na=False)]

# Remove rows with "Expiration" in the 'Description' column
df = df[df['Description'].str.contains('Expiration', na=False)]

# Remove rows with "Cash" in the 'Description' column
df = df[~df['Description'].str.contains('Cash', na=False)]

# Remove rows with "Stock" in the 'Description' column
df = df[~df['Description'].str.contains('Stock', na=False)]

# Ensure 'Activity Date' is in datetime format
df['Activity Date'] = pd.to_datetime(df['Activity Date'], errors='coerce')

# Check for NaT values in 'Activity Date'
if df['Activity Date'].isna().any():
    print("Warning: Some 'Activity Date' values could not be converted to datetime.")

# Sort the DataFrame by 'Instrument' and 'Activity Date' (oldest first)
df.sort_values(by=['Instrument', 'Activity Date'], ascending=[True, True], inplace=True)

# Clean the 'Amount' column and convert to float
def clean_amount(row):
    amount_str = str(row['Amount']).replace('$', '').replace(',', '').strip()  # Clean the amount
    if '(' in amount_str:  # Check for debit
        amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
        return -float(amount_str)  # Return negative for debits
    else:
        return float(amount_str)  # Return positive for credits

# Apply the function to clean the 'Amount' column
df['Amount'] = df.apply(clean_amount, axis=1)
total_amount = df['Amount'].sum() 
print(f'Total Amount: ${total_amount:.2f}') 

# Save the resulting DataFrame to a new CSV file
df.to_csv('../output/filtered_rh_statement.csv', index=False)

print("Filtered, sorted, and updated data saved to filtered_rh_statement.csv")

Total Amount: $-2053.97
Filtered, sorted, and updated data saved to filtered_rh_statement.csv


In [7]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('../output/rh_statement.csv', on_bad_lines='skip')


# Clean the 'Amount' column and convert to float
def clean_amount(row):
    amount_str = str(row['Amount']).replace('$', '').replace(',', '').strip()  # Clean the amount
    if '(' in amount_str:  # Check for debit
        amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
        return -float(amount_str)  # Return negative for debits
    else:
        return float(amount_str)  # Return positive for credits

# Apply the function to clean the 'Amount' column
# df['Amount'] = df.apply(clean_amount, axis=1)
# total_amount = df['Amount'].sum() 
# print(f'Total Amount: ${total_amount:.2f}') 

# Sum the amounts for rows with "ACH" in the 'Description' column
ach_rows = df[df['Description'].str.contains('ACH', na=False)]
ach_total = ach_rows['Amount'].sum()
print(f'Total Amount for ACH transactions: ${ach_total:.2f}')

# Ensure 'Activity Date' is in datetime format
ach_rows['Activity Date'] = pd.to_datetime(ach_rows['Activity Date'], errors='coerce')

# Filter for rows with 'Activity Date' in the year 2024
df_2024 = ach_rows[ach_rows['Activity Date'].dt.year == 2024]
total_2024_amount = df_2024['Amount'].sum()
print(f'Total Amount for the year 2024: ${total_2024_amount:.2f}')

# Save the resulting DataFrame to a new CSV file
ach_rows.to_csv('../output/ach_transactions.csv', index=False)

# print("Filtered, sorted, and updated data saved to filtered_rh_statement.csv")

Total Amount: $564.98
Total Amount for ACH transactions: $4997.00
Total Amount for the year 2024: $7000.00


/tmp/ipykernel_323048/2731824426.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ach_rows['Activity Date'] = pd.to_datetime(ach_rows['Activity Date'], errors='coerce')


In [22]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('../output/rh_statement.csv', on_bad_lines='skip')



# Remove rows with "Expiration" in the 'Description' column
df = df[df['Description'].str.contains('Exercise|Assigned', na=False)]
# Add rows where description contains "Assigned"



#print(df)

# Clean the 'Amount' column and convert to float
def clean_amount(row):
    amount_str = str(row['Amount']).replace('$', '').replace(',', '').strip()  # Clean the amount
    if '(' in amount_str:  # Check for debit
        amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
        return -float(amount_str)  # Return negative for debits
    else:
        return float(amount_str)  # Return positive for credits

# Apply the function to clean the 'Amount' column
df['Amount'] = df.apply(clean_amount, axis=1)
total_amount = df['Amount'].sum() 
print(f'Total Amount: ${total_amount:.2f}') 

df.to_csv("../output/rh_options_assigned_exercised.csv" , index=True)

Total Amount: $-9115.66
